## 5_MAKE_train_test_data
전처리가 완료된 데이터로부터 train과 test set을 만드는 코드입니다.

## 기본 설정

In [1]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

! pip install {USER_FLAG} --upgrade google-cloud-aiplatform
! pip install {USER_FLAG} --upgrade google-cloud-storage
!pip install wget

In [2]:
# 설치가 끝나면 커널을 재시작해야합니다.
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [1]:
# 세팅을 다시 로드
import os
import pandas as pd

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
import warnings
import os
import joblib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from google.cloud import aiplatform, storage

from google.cloud import storage
from google.cloud import bigquery

import pandas as pd
from sklearn import datasets

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
    
# 지역, GCS 클라이언트, BQ 클라이언트 설정
REGION = 'asia-northeast3'
gcs = storage.Client(project = PROJECT_ID)
bq = bigquery.Client(project = PROJECT_ID)

# bucket 이름 지정: "프로젝트 이름-bucket"
BUCKET_NAME=PROJECT_ID + "-bucket"
if not gcs.lookup_bucket(BUCKET_NAME):
    bucketDef = gcs.bucket(BUCKET_NAME)
    bucket = gcs.create_bucket(bucketDef, project=PROJECT_ID, location=REGION)
    print(bucket)
else:
    print(gcs.lookup_bucket(BUCKET_NAME))
    
def get_dataframe(bq_table):
    
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    project_number = shell_output[0]
    bqclient = bigquery.Client(project=project_number)
    
    table = bigquery.TableReference.from_string(
        project_number+"."+bq_table
    )
    
    rows = bqclient.list_rows(
        table
    )
    
    dataframe = rows.to_dataframe(
        create_bqstorage_client=True,
    )

    return dataframe
def to_bigquery(
    dataframe,
    table_name: str,
    bucket_name: str
):
    client = bigquery.Client()
    column_list=dataframe.columns
    query_column_list=""
    
    #blob = storage.blob.Blob.from_string(f"""gs://{bucket_name}/{table_name.split(".")[-1]}.csv""", client=storage.Client())
    dataframe.to_csv(f"""gs://{bucket_name}/{table_name.split(".")[-1]}.csv""",index=False,encoding="utf-8")
    #blob.upload_from_file(temp)
    
    for name in column_list:
        query_column_list=query_column_list+name+" STRING,"
        
    query = f"""
    LOAD DATA OVERWRITE {table_name}(
    {query_column_list}
        )
    FROM FILES (
      skip_leading_rows=1,
      allow_jagged_rows=true,
      format = 'CSV',
      encoding='utf-8',
      field_delimiter=',',
      uris = ['gs://{bucket_name}/{table_name.split(".")[-1]}.csv']);"""
    
    query_job = client.query(query)
    results = query_job.result()

Project ID:  bigfesta2022-esg-team6-proj
<Bucket: bigfesta2022-esg-team6-proj-bucket>


In [32]:
!pip install imblearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 28.1 MB/s eta 0:00:00


## train, test 데이터 

In [3]:
environ_final=pd.read_csv("data/pipeline3/esg_environ_final_cat.csv")
govern_final=pd.read_csv("data/pipeline3/esg_govern_final_cat.csv")
social_final=pd.read_csv("data/pipeline3/esg_social_final_cat.csv")
# or
# environ_final=pd.read_csv("esg_data.esg_environ_final_cat")
# govern_final=pd.read_csv("esg_data.esg_govern_final_cat")
# social_final=pd.read_csv("esg_data.esg_social_final_cat")

In [4]:
esg_train_y=pd.read_csv("data/final_esg/esg_train_data.csv")
esg_test_y=pd.read_csv("data/final_esg/esg_test_data.csv")

### 1. E train, test 데이터

In [5]:
environ_final['industry_med_code']=pd.to_numeric(environ_final['industry_med_code'].str.replace("'",""))
environ_final['industry_med_code']=environ_final['industry_med_code'].fillna(0)

In [6]:
# 학습에 필요한 열만 고르기!
environ_final.columns

Index(['code', 'name', 'market', 'stock_corp_code', 'stock_name', 'kor_name',
       'eng_name', 'industry_code', 'industry_med_code', 'industry_sub_code',
       'industry_med_name', 'industry_sub_name', 'asset_sum', 'raw', 'water',
       'energy', 'air_pollu', 'water_pollu', 'waste', 'chemical',
       'water_reuse', 'waste_reuse', 'exist', 'asset_sum_cat'],
      dtype='object')

In [7]:
# train, test 병합 후 split
environ_train=pd.merge(esg_train_y[["종목코드","E_GRADE"]],environ_final,left_on="종목코드",right_on="code",how="left")
environ_test=pd.merge(esg_test_y[["종목코드","E_GRADE"]],environ_final,left_on="종목코드",right_on="code",how="left")

In [8]:
environ_train.columns
# 'E_GRADE','industry_med_code', 'raw', 'water', 'energy', 'air_pollu','water_pollu', 'waste', 'chemical', 'water_reuse', 'waste_reuse', 'exist', 'asset_sum_cat'

Index(['종목코드', 'E_GRADE', 'code', 'name', 'market', 'stock_corp_code',
       'stock_name', 'kor_name', 'eng_name', 'industry_code',
       'industry_med_code', 'industry_sub_code', 'industry_med_name',
       'industry_sub_name', 'asset_sum', 'raw', 'water', 'energy', 'air_pollu',
       'water_pollu', 'waste', 'chemical', 'water_reuse', 'waste_reuse',
       'exist', 'asset_sum_cat'],
      dtype='object')

In [9]:
environ_train=environ_train[['E_GRADE', 'industry_med_code','raw', 'water', 'energy', 'air_pollu',
       'water_pollu', 'waste', 'chemical', 'water_reuse', 'waste_reuse',
       'exist', 'asset_sum_cat']]
environ_test=environ_test[['code','industry_med_code','raw', 'water', 'energy', 'air_pollu',
       'water_pollu', 'waste', 'chemical', 'water_reuse', 'waste_reuse',
       'exist', 'asset_sum_cat']]

In [10]:
environ_train["E_GRADE"].value_counts()

9    239
7    112
6     89
5     61
8     51
4     47
3     38
2     24
1      2
Name: E_GRADE, dtype: int64

In [11]:
## SMOTE-Tomek
from imblearn.over_sampling import *
# 10, 50, 100, 150, 200 단위로 늘리되, 기존 데이터 수 두 배 한수를 기준으로! 
sampling_strategy = {1: 30, 2: 70,3: 70,4: 120,5: 120,6: 170,7:170,8:120}
X_E_train, Y_E_train = RandomOverSampler(random_state=0,sampling_strategy=sampling_strategy).fit_resample(environ_train.drop("E_GRADE",axis=1),environ_train["E_GRADE"])
environ_train=pd.concat([X_E_train, Y_E_train],axis=1)

In [12]:
Y_E_train.value_counts()

9    239
6    170
7    170
4    120
8    120
5    120
2     70
3     70
1     30
Name: E_GRADE, dtype: int64

In [13]:
environ_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1109 entries, 0 to 1108
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   industry_med_code  1109 non-null   float64
 1   raw                1109 non-null   float64
 2   water              1109 non-null   float64
 3   energy             1109 non-null   float64
 4   air_pollu          1109 non-null   float64
 5   water_pollu        1109 non-null   float64
 6   waste              1109 non-null   float64
 7   chemical           1109 non-null   float64
 8   water_reuse        1109 non-null   float64
 9   waste_reuse        1109 non-null   float64
 10  exist              1109 non-null   int64  
 11  asset_sum_cat      1109 non-null   int64  
 12  E_GRADE            1109 non-null   int64  
dtypes: float64(10), int64(3)
memory usage: 112.8 KB


In [14]:
environ_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284 entries, 0 to 283
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   code               284 non-null    object 
 1   industry_med_code  284 non-null    float64
 2   raw                284 non-null    float64
 3   water              284 non-null    float64
 4   energy             284 non-null    float64
 5   air_pollu          284 non-null    float64
 6   water_pollu        284 non-null    float64
 7   waste              284 non-null    float64
 8   chemical           284 non-null    float64
 9   water_reuse        284 non-null    float64
 10  waste_reuse        284 non-null    float64
 11  exist              284 non-null    int64  
 12  asset_sum_cat      284 non-null    int64  
dtypes: float64(10), int64(2), object(1)
memory usage: 31.1+ KB


In [15]:
## train, test set 내보내기
environ_train.to_csv("data/pipeline3/modeling_data/E_train.csv",index=False,encoding="utf-8")
environ_test.to_csv("data/pipeline3/modeling_data/E_test.csv",index=False,encoding="utf-8")

### 2. S train, test 데이터

In [16]:
social_final['industry_med_code']=pd.to_numeric(social_final['industry_med_code'].str.replace("'",""))
social_final['industry_med_code']=social_final['industry_med_code'].fillna(0)

In [17]:
social_train=pd.merge(esg_train_y[["종목코드","S_GRADE"]],social_final,left_on="종목코드",right_on="code",how="left")
social_test=pd.merge(esg_test_y[["종목코드","S_GRADE"]],social_final,left_on="종목코드",right_on="code",how="left")

In [18]:
social_final.columns

Index(['code', 'name', 'market', 'kor_name', 'eng_name', 'stock_corp_code',
       'stock_name', 'industry_code', 'industry_med_code', 'industry_sub_code',
       'industry_med_name', 'industry_sub_name', 'reg_prob', 'female_prob',
       'reg_female_prob', 'female_money_prob', 'exist', 'disabled',
       'industry_accident', 'volunteer', 'asset_sum', 'asset_sum_cat', 'sm'],
      dtype='object')

In [19]:
social_train=social_train[['S_GRADE', 'industry_med_code','reg_prob', 'female_prob',
       'reg_female_prob', 'female_money_prob', 'exist', 'disabled',
       'industry_accident', 'volunteer', 'asset_sum_cat']]
social_test=social_test[['code', 'industry_med_code','reg_prob', 'female_prob',
       'reg_female_prob', 'female_money_prob', 'exist', 'disabled',
       'industry_accident', 'volunteer', 'asset_sum_cat']]

In [20]:
social_train["S_GRADE"].value_counts()

8    176
9    163
7     97
5     48
6     44
3     42
2     41
4     37
1     15
Name: S_GRADE, dtype: int64

In [21]:
## SMOTE-Tomek
from imblearn.over_sampling import *
# 10, 50, 100, 150, 200 단위로 늘리되, 기존 데이터 수 두 배 한수를 기준으로! 
sampling_strategy = {1: 50, 2: 120,3: 120,4: 70,5: 120,6: 120,7:170}
X_S_train, Y_S_train = RandomOverSampler(random_state=0,sampling_strategy=sampling_strategy).fit_resample(social_train.drop("S_GRADE",axis=1),social_train["S_GRADE"])
social_train=pd.concat([X_S_train, Y_S_train],axis=1)

In [22]:
Y_S_train.value_counts()

8    176
7    170
9    163
3    120
2    120
6    120
5    120
4     70
1     50
Name: S_GRADE, dtype: int64

In [23]:
social_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1109 entries, 0 to 1108
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   industry_med_code  1109 non-null   float64
 1   reg_prob           1109 non-null   float64
 2   female_prob        1109 non-null   float64
 3   reg_female_prob    1109 non-null   float64
 4   female_money_prob  1109 non-null   float64
 5   exist              1109 non-null   bool   
 6   disabled           1109 non-null   float64
 7   industry_accident  1109 non-null   float64
 8   volunteer          1109 non-null   float64
 9   asset_sum_cat      1109 non-null   int64  
 10  S_GRADE            1109 non-null   int64  
dtypes: bool(1), float64(8), int64(2)
memory usage: 87.8 KB


In [24]:
social_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284 entries, 0 to 283
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   code               284 non-null    object 
 1   industry_med_code  284 non-null    float64
 2   reg_prob           284 non-null    float64
 3   female_prob        284 non-null    float64
 4   reg_female_prob    284 non-null    float64
 5   female_money_prob  284 non-null    float64
 6   exist              284 non-null    bool   
 7   disabled           284 non-null    float64
 8   industry_accident  284 non-null    float64
 9   volunteer          284 non-null    float64
 10  asset_sum_cat      284 non-null    int64  
dtypes: bool(1), float64(8), int64(1), object(1)
memory usage: 24.7+ KB


In [25]:
## train, test set 내보내기
social_train.to_csv("data/pipeline3/modeling_data/S_train.csv",index=False,encoding="utf-8")
social_test.to_csv("data/pipeline3/modeling_data/S_test.csv",index=False,encoding="utf-8")

### 3. G train, test 데이터

In [26]:
govern_final['industry_med_code']=pd.to_numeric(govern_final['industry_med_code'].str.replace("'",""))
govern_final['industry_med_code']=govern_final['industry_med_code'].fillna(0)

In [27]:
govern_train=pd.merge(esg_train_y[["종목코드","G_GRADE"]],govern_final,left_on="종목코드",right_on="code",how="left")
govern_test=pd.merge(esg_test_y[["종목코드","G_GRADE"]],govern_final,left_on="종목코드",right_on="code",how="left")

In [28]:
govern_final.columns

Index(['code', 'name', 'market', 'kor_name', 'eng_name', 'stock_corp_code',
       'stock_name', 'industry_code', 'industry_med_code', 'industry_sub_code',
       'industry_med_name', 'industry_sub_name', 'Outer_percentage',
       'female_percentage', 'vote_y', 'vote_n', 'vote_percentage',
       'attendance', 'op_uniformity', 'audit_num', 'exist', 'asset_sum',
       'asset_sum_cat'],
      dtype='object')

In [29]:
govern_train=govern_train[['G_GRADE', 'industry_med_code','Outer_percentage',
       'female_percentage',  'vote_percentage',
       'attendance', 'op_uniformity', 'audit_num', 'exist', 
       'asset_sum_cat']]
govern_test=govern_test[['code','industry_med_code','Outer_percentage',
       'female_percentage',  'vote_percentage',
       'attendance', 'op_uniformity', 'audit_num', 'exist', 
       'asset_sum_cat']]

In [30]:
govern_train["G_GRADE"].value_counts()

3    82
8    82
1    79
9    77
7    75
4    70
5    69
2    65
6    64
Name: G_GRADE, dtype: int64

In [31]:
## SMOTE-Tomek
from imblearn.over_sampling import *
# 10, 50, 100, 150, 200 단위로 늘리되, 기존 데이터 수 두 배 한수를 기준으로! 
sampling_strategy = {1: 120, 2:120,3:120,4: 120,5: 120,6: 120,7: 120,8: 120,9: 120}
X_G_train, Y_G_train = RandomOverSampler(random_state=0).fit_resample(govern_train.drop("G_GRADE",axis=1),govern_train["G_GRADE"])
govern_train=pd.concat([X_G_train, Y_G_train],axis=1)

In [32]:
Y_G_train.value_counts()

2    82
1    82
3    82
6    82
4    82
8    82
5    82
7    82
9    82
Name: G_GRADE, dtype: int64

In [33]:
extra_govern=govern_train.sample(frac=0.5,random_state=1004).reset_index(drop=True)
govern_train=pd.concat([govern_train,extra_govern],axis=0)
govern_train=govern_train.reset_index(drop=True)

In [34]:
govern_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1107 entries, 0 to 1106
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   industry_med_code  1107 non-null   float64
 1   Outer_percentage   1107 non-null   float64
 2   female_percentage  1107 non-null   float64
 3   vote_percentage    1107 non-null   float64
 4   attendance         1107 non-null   float64
 5   op_uniformity      1107 non-null   float64
 6   audit_num          1107 non-null   float64
 7   exist              1107 non-null   bool   
 8   asset_sum_cat      1107 non-null   int64  
 9   G_GRADE            1107 non-null   int64  
dtypes: bool(1), float64(7), int64(2)
memory usage: 79.0 KB


In [35]:
govern_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284 entries, 0 to 283
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   code               284 non-null    object 
 1   industry_med_code  284 non-null    float64
 2   Outer_percentage   284 non-null    float64
 3   female_percentage  284 non-null    float64
 4   vote_percentage    284 non-null    float64
 5   attendance         284 non-null    float64
 6   op_uniformity      284 non-null    float64
 7   audit_num          284 non-null    float64
 8   exist              284 non-null    bool   
 9   asset_sum_cat      284 non-null    int64  
dtypes: bool(1), float64(7), int64(1), object(1)
memory usage: 22.5+ KB


In [36]:
## train, test set 내보내기
govern_train.to_csv("data/pipeline3/modeling_data/G_train.csv",index=False,encoding="utf-8")
govern_test.to_csv("data/pipeline3/modeling_data/G_test.csv",index=False,encoding="utf-8")

### 4. ESG train, test 데이터

In [37]:
environ_final=pd.read_csv("data/pipeline3/esg_environ_final_cat.csv")
govern_final=pd.read_csv("data/pipeline3/esg_govern_final_cat.csv")
social_final=pd.read_csv("data/pipeline3/esg_social_final_cat.csv")

In [38]:
ESG_final=pd.merge(environ_final,govern_final,on="code")
ESG_final=pd.merge(ESG_final,social_final,on="code")

In [39]:
ESG_final['industry_med_code']=pd.to_numeric(ESG_final['industry_med_code'].str.replace("'",""))
ESG_final['industry_med_code']=ESG_final['industry_med_code'].fillna(0)

In [40]:
ESG_train=pd.merge(esg_train_y[["종목코드","ESG_GRADE"]],ESG_final,left_on="종목코드",right_on="code",how="left")
ESG_test=pd.merge(esg_test_y[["종목코드","ESG_GRADE"]],ESG_final,left_on="종목코드",right_on="code",how="left")

In [41]:
ESG_train.columns

Index(['종목코드', 'ESG_GRADE', 'code', 'name_x', 'market_x', 'stock_corp_code_x',
       'stock_name_x', 'kor_name_x', 'eng_name_x', 'industry_code_x',
       'industry_med_code_x', 'industry_sub_code_x', 'industry_med_name_x',
       'industry_sub_name_x', 'asset_sum_x', 'raw', 'water', 'energy',
       'air_pollu', 'water_pollu', 'waste', 'chemical', 'water_reuse',
       'waste_reuse', 'exist_x', 'asset_sum_cat_x', 'name_y', 'market_y',
       'kor_name_y', 'eng_name_y', 'stock_corp_code_y', 'stock_name_y',
       'industry_code_y', 'industry_med_code_y', 'industry_sub_code_y',
       'industry_med_name_y', 'industry_sub_name_y', 'Outer_percentage',
       'female_percentage', 'vote_y', 'vote_n', 'vote_percentage',
       'attendance', 'op_uniformity', 'audit_num', 'exist_y', 'asset_sum_y',
       'asset_sum_cat_y', 'name', 'market', 'kor_name', 'eng_name',
       'stock_corp_code', 'stock_name', 'industry_code', 'industry_med_code',
       'industry_sub_code', 'industry_med_name', 'in

In [42]:
ESG_train=ESG_train[['ESG_GRADE', 'industry_med_code','asset_sum_cat','raw', 'water', 'energy', 'air_pollu',
       'water_pollu', 'waste', 'chemical', 'water_reuse', 'waste_reuse','Outer_percentage', 'female_percentage','vote_percentage', 'attendance', 'op_uniformity','audit_num','reg_female_prob', 'female_money_prob', 'exist', 'disabled',
       'industry_accident', 'volunteer','exist_x','exist_y']]
ESG_test=ESG_test[['code','industry_med_code','asset_sum_cat','raw', 'water', 'energy', 'air_pollu',
       'water_pollu', 'waste', 'chemical', 'water_reuse', 'waste_reuse','Outer_percentage', 'female_percentage','vote_percentage', 'attendance', 'op_uniformity','audit_num','reg_female_prob', 'female_money_prob', 'exist', 'disabled',
       'industry_accident', 'volunteer','exist_x','exist_y']]

In [43]:
ESG_train["exist_total"]=ESG_train["exist"]+ESG_train["exist_x"]+ESG_train["exist_y"]
ESG_test["exist_total"]=ESG_test["exist"]+ESG_test["exist_x"]+ESG_test["exist_y"]
ESG_train=ESG_train.drop(["exist","exist_x","exist_y"],axis=1)
ESG_test=ESG_test.drop(["exist","exist_x","exist_y"],axis=1)

In [44]:
ESG_train["ESG_GRADE"].value_counts()

8    214
7    134
9     82
5     59
6     53
4     46
3     40
2     29
1      6
Name: ESG_GRADE, dtype: int64

In [45]:
## SMOTE-Tomek
from imblearn.over_sampling import *
# 10, 50, 100, 150, 200 단위로 늘리되, 기존 데이터 수 두 배 한수를 기준으로! 
sampling_strategy = {1: 30, 2: 70,3: 70,4: 120,5: 120,6: 120,7:170,9:170}
X_ESG_train, Y_ESG_train = RandomOverSampler(random_state=0,sampling_strategy=sampling_strategy).fit_resample(ESG_train.drop("ESG_GRADE",axis=1),ESG_train["ESG_GRADE"])
ESG_train=pd.concat([X_ESG_train, Y_ESG_train],axis=1)

In [46]:
Y_ESG_train.value_counts()

8    214
7    170
9    170
4    120
6    120
5    120
3     70
2     70
1     30
Name: ESG_GRADE, dtype: int64

In [47]:
## train, test set 내보내기
ESG_train.to_csv("data/pipeline3/modeling_data/ESG_train.csv",index=False,encoding="utf-8")
ESG_test.to_csv("data/pipeline3/modeling_data/ESG_test.csv",index=False,encoding="utf-8")

In [48]:
ESG_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1084 entries, 0 to 1083
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   industry_med_code  1084 non-null   float64
 1   asset_sum_cat      1084 non-null   int64  
 2   raw                1084 non-null   float64
 3   water              1084 non-null   float64
 4   energy             1084 non-null   float64
 5   air_pollu          1084 non-null   float64
 6   water_pollu        1084 non-null   float64
 7   waste              1084 non-null   float64
 8   chemical           1084 non-null   float64
 9   water_reuse        1084 non-null   float64
 10  waste_reuse        1084 non-null   float64
 11  Outer_percentage   1084 non-null   float64
 12  female_percentage  1084 non-null   float64
 13  vote_percentage    1084 non-null   float64
 14  attendance         1084 non-null   float64
 15  op_uniformity      1084 non-null   float64
 16  audit_num          1084 

In [49]:
ESG_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284 entries, 0 to 283
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   code               284 non-null    object 
 1   industry_med_code  284 non-null    float64
 2   asset_sum_cat      284 non-null    int64  
 3   raw                284 non-null    float64
 4   water              284 non-null    float64
 5   energy             284 non-null    float64
 6   air_pollu          284 non-null    float64
 7   water_pollu        284 non-null    float64
 8   waste              284 non-null    float64
 9   chemical           284 non-null    float64
 10  water_reuse        284 non-null    float64
 11  waste_reuse        284 non-null    float64
 12  Outer_percentage   284 non-null    float64
 13  female_percentage  284 non-null    float64
 14  vote_percentage    284 non-null    float64
 15  attendance         284 non-null    float64
 16  op_uniformity      284 non

In [ ]:
## train, test set 내보내기
to_bigquery(environ_train."esg_data.E_train",BUCKET_NAME)
to_bigquery(environ_test."esg_data.E_test",BUCKET_NAME)
## train, test set 내보내기
to_bigquery(social_train."esg_data.S_train",BUCKET_NAME)
to_bigquery(socoial_test."esg_data.S_test",BUCKET_NAME)
## train, test set 내보내기
to_bigquery(ESG_train."esg_data.ESG_train",BUCKET_NAME)
to_bigquery(ESG_test."esg_data.ESG_test",BUCKET_NAME)
## train, test set 내보내기
to_bigquery(govern_train."esg_data.G_train",BUCKET_NAME)
to_bigquery(govern_test."esg_data.G_test",BUCKET_NAME)